# STAT 301 Final Group Report

# Title

Group Members:
- Vivaan Jhaveri (39723044)
- Ethan Rajkumar
- Michael Wang
- Ruhani Kaur

## Introduction

Employee turnover is a critical challenge for organizations as it directly impacts productivity, morale, and financial performance. High turnover rates can lead to increased recruitment and training costs, loss of institutional knowledge, and disruptions to team dynamics. Understanding the factors that drive employees to leave is essential for developing effective retention strategies. Variables such as compensation (payment tier), job satisfaction (benching status), demographic characteristics (age, gender), and professional qualifications (education, experience) often play significant roles in influencing turnover. By leveraging data-driven methods like logistic regression, organizations can identify key predictors of employee turnover and quantify their impact, enabling proactive interventions to enhance workforce stability.

The Employees dataset provides anonymized data on company employees, including details such as educational attainment, job tenure, office location, salary tier, demographic attributes, and their work status within the organization. This dataset was gathered by the company’s Human Resources team and subsequently shared on Kaggle, a platform that hosts datasets and resources for data science professionals and enthusiasts.

Key Statistics:
- Total Observations: 4,653 employees
- Total Variables: 9 variables, each contributing to potential insights on employee demographics, job role characteristics, and retention patterns.

Key Variables:
- `Education`: Categorical variable representing the highest level of education attained by the employee ("Bachelors" "Masters", "PhD").
- `JoiningYear`: Numerical variable representing the year the employee joined the company.
- `City`: Categorical variable representing the city where the employee is located ("New Delhi", "Bangalore" "Pune").
- `PaymentTier`: Categorical variable representing the different salary tiers (1, 2, 3).
- `Age`: Numerical variable representing the age of the employee.
- `Gender`: Categorical variable representing the gender of the employee ("Male", "Female").
- `EverBenched`: Categorical (binary) variable representing whether the employee has ever been "benched" ("Yes") or not ("No").
- `ExperienceInCurrentDomain`: Numerical variable representing years of experience the employee has.
- `LeaveOrNot`: Binary response variable representing whether the employee left the company (1) or stayed (0).

This dataset offers a mix of demographic, employment, and behavioral information. Each record captures unique attributes for individual employees, enabling analysis of factors that may contribute to whether an employee stays with or leaves the company. Insights drawn from this data can help organizations understand key drivers of employee retention and guide HR policy improvements.

[Link to the Dataset](https://www.kaggle.com/datasets/tawfikelmetwally/employee-dataset/data)


## Question
Which specific factors, such as age, payment tier, benching status, and experience, are statistically significant predictors of whether an employee will leave the organization? Additionally, how does each factor impact the likelihood of turnover?

Additionally, align your question/objectives with the existing literature. To contextualize your study, include a minimum of two scientific publications (these should be listed in the References section).

## Methods and Results

### Exploratory Data Analysis (EDA)

Demonstrate that the dataset can be read into R.
Clean and wrangle your data into a tidy format.
Plot the relevant raw data, tailoring your plot to address your question.
Make sure to explore the association of the explanatory variables with the response.
Any summary tables that are relevant to your analysis.
Be sure not to print output that takes up a lot of screen space.
Your EDA must be comprehensive with high quality plots.

### Methods: Plan

Describe in written English the methods you used to perform your analysis from beginning to end, and narrate the code that does the analysis.
If included, describe the “Feature Selection” process and how and why you choose the covariates of your final model.
Make sure to interpret/explain the results you obtain. It’s not enough to just say, “I fitted a linear model with these covariates, and my R-square is 0.87”.
If inference is the aim of your project, a detailed interpretation of your fitted model is required, as well as a discussion of relevant quantities (e.g., are the coefficients significant? How does the model fit the data)?
A careful model assessment must be conducted.
If prediction is the project's aim, describe the test data used or how it was created.
Ensure your tables and/or figures are labelled with a figure/table number.

## Discussion

In this section, you’ll interpret the results you obtained in the previous section with respect to the main question/goal of your project.

Summarize what you found and the implications/impact of your findings.
If relevant, discuss whether your results were what you expected to find.
Discuss how your model could be improved;
Discuss future questions/research this study could lead to.

## References

At least two citations of literature relevant to the project. The citation format is your choice – just be consistent. Make sure to cite the source of your data as well.